## Tik Tok

In [64]:
import os
import numpy as np
from py2neo import Graph
from py2neo import Node
from py2neo import Relationship
import pandas as pd
import numpy as np
# for some nlp secret sauce
import nltk
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from langdetect import detect_langs
from langdetect import detect
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [15]:
# Run the next two lines as needed
#nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\peter\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [19]:
#nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\peter\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [21]:
# update bolt port number and password as needed.
# in Neo4j, start DB of choice, click on DB name, on right under Details tab note 'Bolt port'
# Windows
if os.name == 'nt':
    graph = Graph("bolt://localhost:7687", password='xxx', name='neo4j')
# Linux & Mac
if os.name == 'posix':
    graph = Graph("bolt://localhost:11003", password='xxx', name='neo4j')

In [22]:
# I don't think this needs to be run every time
query = """
CREATE INDEX USRIDX IF NOT EXISTS
FOR (n:user)
ON (n.id);
"""
graph.run(query)

(No data)

In [23]:
# Quick test to make sure things are connected and working. This Okean Elzy song should show up twice.
query = """
MATCH (s:song) WHERE s.title =~ '.*Обійми.*' RETURN s;
"""
graph.run(query)

s                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 (_342470:song {album: '\u0417\u0435\u043c\u043b\u044f', audioURLWithCookie: 0, authorName: '\u041e\u043a\u0435\u0430\u043d \u0415\u043b\u044c\u0437\u0438', coverLarge: 'https://p16-amd-va.tiktokcdn.com/img/tos-useast2a-v-2774/0e057b52e54948e8a5d6ebc686ef8e3b~c5_720x720.jpeg', coverMedium: 'https://p16-amd-va.tiktokcdn.com/img/tos-useast2a-v-2774/0e057b52e54948e8a5d6ebc686ef8e3b~c5_200x200.jpeg', coverThumb: 'https://p16-amd-va.tiktokcdn.com/img/tos-useast2a-v-2774/0e057b52e54948e8a5d6ebc686ef8e3b~c5_100x100.jpeg', duration: 60, id: '6718412074684254210', keyToken: '', numVids: 19300, original: 0, playUrl: 'https://sf16-ies-music-va.tiktokcdn.com/obj/tos-useast2a-ve-2774/76156347b51b4cbaa836c9e444e5872b', private: 0, scheduleSearchTime: 0, title: '\u041e\u0431\u0456\u0439\u043c\u0438'})                        
 (_343278:song {album: '\u041e\u0431\u0456\u0439\u043c\u0438 - Remix', audioURLWithCookie: 0, authorName: '\u041e\u043a\u0435\u0430\u043d \u0415\u043b\u044c\u0437\u0438', coverLarge: 'https://p16-amd-va.tiktokcdn.com/img/tos-useast2a-v-2774/60a559e85efd4e40877e660f651efdfd~c5_720x720.jpeg', coverMedium: 'https://p16-amd-va.tiktokcdn.com/img/tos-useast2a-v-2774/60a559e85efd4e40877e660f651efdfd~c5_200x200.jpeg', coverThumb: 'https://p16-amd-va.tiktokcdn.com/img/tos-useast2a-v-2774/60a559e85efd4e40877e660f651efdfd~c5_100x100.jpeg', duration: 60, id: '6899164801193216002', keyToken: '', numVids: '6733', original: 0, playUrl: 'https://sf16-ies-music-va.tiktokcdn.com/obj/tos-useast2a-ve-2774/f2e8a7defbd44b08858d6c6d04eefc97', private: 0, scheduleSearchTime: 0, title: '\u041e\u0431\u0456\u0439\u043c\u0438 - Remix'})

In [6]:
# Search for Okean Elzy. This is the biggest Ukranian rock band. They sellout football stadiums in Ukraine.
query = """
MATCH (s:song) WHERE s.authorName =~ '.*Океан Ельзи.*' 
RETURN s.title AS title, s.album AS album;
"""
graph.run(query)

title         | album   
---------------|---------
 Не твоя війна | Без меж 
 Без бою       | Gloria  
 Обійми        | Земля

In [7]:
# Search for Time & Glass. Время и Стекло are a poppy/dancy group.
query = """
MATCH (s:song) WHERE s.authorName =~ '.*Время и Стекло.*' 
RETURN s.title AS title, s.album AS album;
"""
graph.run(query)

(No data)

In [8]:
# Search for Time
query = """
MATCH (s:song) WHERE s.authorName =~ '.*Время.*' 
RETURN s.title AS title, s.album AS album;
"""
graph.run(query)

(No data)

In [8]:
# How about their song Тролль? Really, 187M views on youtube and squat here?
query = """
MATCH (s:song) WHERE s.title =~ '.*Тролль.*' RETURN s;
"""
graph.run(query)

(No data)

In [9]:
query = """
MATCH (u:user)
WITH u, SIZE(()-[:recommends]->(u)) as recommendCnt
RETURN u.id AS username,
       recommendCnt
ORDER BY recommendCnt DESC
"""
graph.run(query)

username       | recommendCnt 
----------------|--------------
 maliciadorable |          151 
 nicoenvrai     |          149 
 celinedept     |          148

## Centrality Measures
<A HREF="https://neo4j-website.s3.eu-central-1.amazonaws.com/build/html/Algorithms/centrality/centrality.html">Centrality Measures</A>

First, install <A HREF="https://neo4j.com/labs/apoc/4.1/installation/">APOC</A> and Graph Data Science Library in the Tik Tok database

In [10]:
# Show graph structure, works better in Neo4j browser
query = """
CALL db.schema.visualization()
"""
graph.run(query)

nodes                                                                                                                                                                                                                                    | relationships                                                                                                                                                                                                                      
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 [(_-8:song {constraints: [], indexes: [], name: 'song'}), (_-5:video {constraints: [], indexes: [], name: 'video'}), (_-7:tag {constraints: [], indexes: [], name: 'tag'}), (_-6:user {constraints: [], indexes: ['id'], name: 'user'})] | [(_-6)-[:author {}]->(_-8), (_-6)-[:author {}]->(_-5), (_-5)-[:has {}]->(_-8), (_-5)-[:has {}]->(_-7), (_-5)-[:recommends {}]->(_-6), (_-6)-[:recommends {}]->(_-5), (_-6)-[:recommends {}]->(_-6), (_-5)-[:recommends {}]->(_-5)]

In [12]:
# ok already did this, doesn't want to be created again
# java.lang.IllegalArgumentException: A graph with name 'users' already exists
query = """
CALL gds.graph.create('users','user','recommends');
"""
#graph.run(query)

In [13]:
# This takes a little bit to run, apparently this graph/database is large
# Counties with highest centrality score
query = """
CALL gds.alpha.closeness.stream({
    nodeProjection: "user",
    relationshipProjection: "recommends"
})
YIELD nodeId, centrality
RETURN gds.util.asNode(nodeId).id AS name, 
       centrality AS centralScore
ORDER BY centrality DESC;
"""
result=graph.run(query)
centrality = result.to_data_frame()
centrality.head(10)

name  centralScore
0          poppucatt           1.0
1           dnscnst5           1.0
2    wass_officiel31           1.0
3  thelondonersstory           1.0
4         romania.go           1.0
5           mari.fil           1.0
6         bxtterf1y2           1.0
7            ripachu           1.0
8   james_cardwell04           1.0
9       casperisdope           1.0

In [ ]:
#centrality.to_csv("centrality.csv", index=False)

In [14]:
# Counties with highest betweeness score
query = """
CALL gds.betweenness.stream({
    nodeProjection: "user",
    relationshipProjection: "recommends"
})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name,
       gds.util.asNode(nodeId).followerCount,
       score AS btwnScore
ORDER BY score DESC;
"""
result=graph.run(query)
betweeness = result.to_data_frame()
betweeness.head(10)

name  gds.util.asNode(nodeId).followerCount     btwnScore
0  thehistorylegends                               610500.0  3.508975e+07
1       themagicmatt                              4200000.0  2.766751e+07
2       jonathan.blu                                    NaN  2.690732e+07
3          kingkhieu                                    NaN  2.670178e+07
4        rarestworld                              1600000.0  2.056806e+07
5          danrhodes                                    NaN  1.976106e+07
6        twiceofnice                             12200000.0  1.838829e+07
7  manchurektriplets                             11600000.0  1.813186e+07
8         dobretwins                             35400000.0  1.557718e+07
9    twinsfromrussia                             10800000.0  1.552047e+07

In [ ]:
#betweeness.to_csv("betweeness.csv", index=False)

In [6]:
# Counties with highest page rank score
query = """
CALL gds.pageRank.stream({  
    nodeProjection: "user",
    relationshipProjection: "recommends",  
    maxIterations: 50,  dampingFactor: 0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).id AS name, 
       gds.util.asNode(nodeId).followerCount AS followerCount,
       score AS pageRank
ORDER BY score DESC;
"""
result=graph.run(query)
pagerank = result.to_data_frame()
pagerank.head(10)

name  followerCount  pageRank
0    maliciadorable      3000000.0  0.924330
1        nicoenvrai      1200000.0  0.917667
2        celinedept     12400000.0  0.912277
3          badfifii      1100000.0  0.896603
4  maudevandervorst       488700.0  0.873413
5     gardianhasani      1800000.0  0.873274
6  helena_la_blonde       896400.0  0.869802
7     sarah_bossuwe       612700.0  0.869145
8    iamshadeandrew      1300000.0  0.869141
9           7futsal      2200000.0  0.853890

In [ ]:
#pagerank.to_csv("pagerank.csv", index=False)

## Community Detection
<A HREF="https://neo4j-website.s3.eu-central-1.amazonaws.com/build/html/Algorithms/community/community.html">Class Notes on Community Detection</A>

<A HREF="https://neo4j.com/docs/graph-data-science/current/algorithms/louvain/">Cypher Louvain Modularity</A><BR>

In [ ]:
# Louvain Modularity 
# Louvain quantifies how well a node is assigned to a group by looking at the density of connections within 
# a cluster in comparison to an average or random sample
query = """
CALL gds.louvain.stream({
  nodeProjection: "County",
  relationshipProjection: "COMMUTES",
  includeIntermediateCommunities: true
})
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).geoid AS geoid, 
       gds.util.asNode(nodeId).county AS county,
       gds.util.asNode(nodeId).state AS state,
       communityId, 
       intermediateCommunityIds
ORDER BY communityId ASC;
"""
result=graph.run(query)
louvainMod = result.to_data_frame()
louvainMod.head(10)

In [ ]:
#louvainMod.to_csv("louvainmod.csv", index=False)

In [17]:
# Let's see if we can find anything on the white house briefing of the top30 TikTok users
# https://www.washingtonpost.com/technology/2022/03/11/tik-tok-ukraine-white-house/
# "National Security Council staffers and White House press secretary Jen Psaki briefed the influencers"
query = """
MATCH (u:user)-[:author]->(v:video)-[:has]->(t:tag)
WHERE t.title =~ '.*Psaki.*'
RETURN u.id
"""
graph.run(query)

(No data)

In [19]:
query = """
MATCH (u:user)-[:author]->(v:video)
WHERE v.desc =~ '.*Psaki.*'
RETURN u.id AS userName
"""
graph.run(query)

userName 
----------
 cbsnews

In [30]:
query = """
MATCH (u:user)-[:author]->(v:video)
WHERE v.desc =~ '.*Biden.*'
RETURN u.id AS userName,
       v.desc AS videoDescription,
       v.playCount AS playCount;
"""
graph.run(query)

userName            | videoDescription                                                                                                                                                                                                                              | playCount 
---------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-----------
 bikini.bottom.news  | President Biden says there will be an upcoming food shortage #news #fyp #foryou #biden #presidentbiden #nato #belgium #natosummit #ukraine #russia #food #foodshortage #war #bikinibottomnews #invasion #unitedstates #usa #europe #sanctions |   9200000 
 nbcnews             | Pres. #Biden: “When Americans and #Russia start shooting at one another, we’re in a very different world than we’ve ever been in.” @NBC Nightly News                                                                                          |  33300000 
 philippine_military | President Biden Send Warships In West Philippine Sea#foryou#fyp#foryoupage#usnavy#us#usa#america#navy#philippines#military#joebiden#nato#trend #viral                                                                                         |   4700000

In [7]:
query = """
MATCH (u:user)-[:author]->(v:video)
WHERE v.desc =~ '.*Croo.*'
RETURN u.id AS userName,
       v.desc AS videoDescription;
"""
graph.run(query)

(No data)

In [ ]:
# Wow, I just don't know how to tik tok

In [34]:
query = """
MATCH (u:user)-[:author]->(v:video)
WHERE v.desc =~ '.*revolution.*'
RETURN u.id AS userName,
       v.desc AS videoDescription,
       v.playCount AS playCount,
       count(v);
"""
graph.run(query)

userName     | videoDescription                                                                                                 | playCount | count(v) 
--------------|------------------------------------------------------------------------------------------------------------------|-----------|----------
 luxy_capi3   | @Capi_revolution calme t’a copine                                                                                |   6400000 |        1 
 sarah.accart | 10 fois bien meilleure 🥰🥰 #caprisun #sirop #revolution                                                           |   1100000 |        1 
 golf.daddy   | Reply to @frankieirontagerbar  the most revolutionary product in divot science ⛳️ #golf #golftiktok #fyp #golfbf |   7200000 |        1

In [52]:
query = """
MATCH (u:user)-[a:author]->(v:video)
WHERE v.desc =~ '.*ukraine.*' AND
      v.stickerText =~ '.*🇺🇦.*'
RETURN u.id AS userName,
       v.desc AS videoDescription,
       v.playCount AS playCount,
       v.stickerText AS sticker
ORDER BY v.playCount DESC;
"""
graph.run(query)

userName       | videoDescription                                                                                               | playCount | sticker                         
----------------|----------------------------------------------------------------------------------------------------------------|-----------|---------------------------------
 foryouukrainee | #ukraine #viralvideo #army #russia #fypシ                                                                       |  48800000 | 🇺🇦STOP WAR 🇷🇺                   
 stopthwar      | STOP THE WAR 🇺🇦🙏🏻 #ukraine #russia #usa #nato #joebiden #biden #vladimirputin #putin #zielinski #war #soldiers |  38600000 | UKRAINE LIVE 😱🇺🇦                
 valerisssh     | Putin, I wait u in Chernihiv 🥰🥰🥰🥰 #ukraine #stopwar                                                            |  21900000 | pov: you are live in Ukraine💙🇺🇦

In [44]:
query = """
MATCH (u:user)-[a:author]->(v:video)
WHERE v.desc =~ '.*slava.*'
RETURN count(v);
"""
graph.run(query)

count(v) 
----------
      346

In [50]:
query = """
MATCH (u:user)-[a:author]->(v:video)
WHERE v.desc =~ '.*slava.*'
RETURN u.id AS name,
       count(a) AS numberVids
ORDER BY numberVids;
"""
graph.run(query)

name            | numberVids 
-----------------|------------
 laceyjanebrown  |          1 
 howtobepositive |          1 
 slouxxie        |          1

In [41]:
query = """
MATCH (u:user)-[a:author]->(v:video)
WHERE v.desc =~ '.*slavaukraini.*'
RETURN u.id AS userName,
       v.desc AS videoDescription,
       v.playCount AS playCount,
       count(*)
ORDER BY v.playCount DESC;
"""
graph.run(query)

userName        | videoDescription                                                                                                                                                                                              | playCount | count(*) 
-----------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-----------|----------
 howtobepositive | Thousands of people gathered together in Toronto to show support to Ukraine 🇺🇦  #standwithukraine #slavaukraini #slavaukraine #supportukraine                                                                 |   1900000 |        1 
 shmeebit        | Reply to @momxietysociety  #greenscreen #sayless #presidentzelensky #slavaukraini🇺🇦 #ukraine #prayforukraine                                                                                                  |   1800000 |        1 
 morriganamber   | Based on true events 👏👏🔥🔥#2022#ukrainewar#stoprussia#stopputin#ukraineinvasion#saveukraine#slavaukraini #war#warship#ww3#navy#military#usa#europe#nato#EU#UK#au#battleship#rocket#history#warhistory#politics |     83300 |        1

In [59]:
# NLP in Neo4j https://neo4j.com/labs/apoc/4.2/nlp/
# Aw man, you gotta sign up for AWS or Azure and they want credit cards, boooo!!!!
# Can't I just do some NLP? Why isn't this built in to Neo4j?
# I want to grab videos with description or stickers in Ukranian, I don't need conglomerate services!

In [ ]:
# hmmm maybe this will work?
# https://www.awseducate.com/Registration?apptype=student&courseview=true#INFO-Student
# Nope, nevermind

In [16]:
sentence = "За чим стоять усі ці люди в центрі Києва?! #україна #ukraine #київ #укрпошта #російськийкорабель #kyiv"
print(detect_langs(sentence))

[uk:0.9999979424399521]


In [17]:
sentence = "Ukrainian band who are serving as soldiers reach out to Ed Sheeran #Antytila #ukraine #ukrainewar #russia #bbcnews #fundraiser #uk"
print(detect_langs(sentence))

[en:0.9999948973920949]


In [9]:
query = """
MATCH (u:user)-[a:author]->(v:video)
WHERE u.id =~ '.*bbcnews.*' and
      v.desc =~ '.*ukraine.*'
RETURN v.desc AS description;
"""
result = pd.DataFrame(graph.run(query).data())
result.head()

description
0  Чи добре їдять українські воїни? Ми попросили ...
1  The Syrians signing up to fight for Russia #uk...
2  Putin hails war with concert and heroism lesso...
3  Children with cancer from Ukraine being helped...
4  What is life in Ukraine like under Russian occ...

In [10]:
ukcount=0
encount=0
for sentence in result['description']:
    lang = detect(sentence)
    if lang=='uk':
        ukcount+=1
    if lang=='en':
        encount+=1
print(encount,"english messages and,", ukcount,"ukranian messages")

18 english messages and, 24 ukranian messages


In [11]:
# polarity score is a float within the range [-1.0, 1.0]
# <0 - negative, sad, >0 - positive, happy
for sentence in result['description']:
    lang = detect(sentence)
    if lang=='en':
        blob = TextBlob(sentence)
        print(round(blob.sentiment.polarity,3)," - ",blob)

0.0  -  The Syrians signing up to fight for Russia #ukrainewar #ukraine #russia #bbcnews
0.0  -  Putin hails war with concert and heroism lessons #russia #ukraine #ukrainewar #bbcnews
0.0  -  Children with cancer from Ukraine being helped in Polish hotel #ukrainewar #poland #ukraine #russia #bbcnews
0.0  -  What is life in Ukraine like under Russian occupation? #ukraine #ukrainewar #russia #bbcnews
0.0  -  Ukrainian band who are serving as soldiers reach out to Ed Sheeran #Antytila #ukraine #ukrainewar #russia #bbcnews #fundraiser #uk
0.0  -  Protesters in Ukraine appear to force Russian truck to reverse#ukraine #ukrainewar #russia #bbcnews
0.0  -  A statue representing friendship between Ukraine and Russia has been toppled in Kyiv. #ukrainewar #russia #ukraine #bbcnews
0.0  -  Russian missile kills baby Kira, her mother and her grandmother - her father Yuriy Glodan talks to BBC News #ukrainewar #russia #bbcnews #ukraine
0.0  -  Ukrainian drones monitoring Russian troops #ukraine #russ

In [12]:
# subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective
for sentence in result['description']:
    lang = detect(sentence)
    if lang=='en':
        blob = TextBlob(sentence)
        print('%.2f'%blob.sentiment.subjectivity)

0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.00
0.53
0.00
0.70
0.00
0.00
0.00


In [18]:
# try NLTK's vader SentimentIntensityAnalyzer
# https://www.nltk.org/api/nltk.sentiment.vader.html?highlight=sentimentintensityanalyzer
# grabbed some code from https://www.programcreek.com/python/example/100005/nltk.sentiment.vader.SentimentIntensityAnalyzer
# note: compound is a normalized sum of neg, neu, pos
sai = SentimentIntensityAnalyzer()
for sentence in result['description']:
    lang = detect(sentence)
    if lang=='en':
        kvp = sai.polarity_scores(sentence)
        for k in kvp:
            print('{} = {} '.format(k, kvp[k]), end='--')
        print('[{}]'.format(sentence))
        print() 

neg = 0.191 --neu = 0.809 --pos = 0.0 --compound = -0.3818 --[The Syrians signing up to fight for Russia #ukrainewar #ukraine #russia #bbcnews]

neg = 0.22 --neu = 0.565 --pos = 0.215 --compound = -0.0258 --[Putin hails war with concert and heroism lessons #russia #ukraine #ukrainewar #bbcnews]

neg = 0.239 --neu = 0.761 --pos = 0.0 --compound = -0.6597 --[Children with cancer from Ukraine being helped in Polish hotel #ukrainewar #poland #ukraine #russia #bbcnews]

neg = 0.0 --neu = 0.828 --pos = 0.172 --compound = 0.3612 --[What is life in Ukraine like under Russian occupation? #ukraine #ukrainewar #russia #bbcnews]

neg = 0.0 --neu = 0.942 --pos = 0.058 --compound = 0.0258 --[Ukrainian band who are serving as soldiers reach out to Ed Sheeran #Antytila #ukraine #ukrainewar #russia #bbcnews #fundraiser #uk]

neg = 0.137 --neu = 0.863 --pos = 0.0 --compound = -0.2263 --[Protesters in Ukraine appear to force Russian truck to reverse#ukraine #ukrainewar #russia #bbcnews]

neg = 0.0 --neu 

In [76]:
sai = SentimentIntensityAnalyzer()
news=['abcnews','bbcnews','latimes','cbsnews','nbcnews','skynews','nypost']
for new in news:
    score=[]
    query = """
    MATCH (u:user)-[a:author]->(v:video)
    WHERE u.id =~ '%s' and
          v.desc =~ '.*ukraine.*'
    RETURN v.desc AS description; 
    """ % new
    # the % new inserts value of new where the %s is in the WHERE statement
    result=pd.DataFrame(graph.run(query).data())
    if (len(result)!=0):
        for sentence in result['description']:
            lang = detect(sentence)
            if lang=='en':
                kvp = sai.polarity_scores(sentence)
                score.append(kvp['compound'])
        print(new,'has mean sentiment score of', round(np.mean(score),3),'from', len(score),'videos')
        print()
    else:
        print(new,'returned no results with ukraine in video description')
        print()

abcnews returned no results with ukraine in video description

bbcnews has mean sentiment score of -0.12 from 18 videos

latimes returned no results with ukraine in video description

cbsnews has mean sentiment score of -0.237 from 7 videos

nbcnews returned no results with ukraine in video description

skynews has mean sentiment score of 0.083 from 2 videos

nypost returned no results with ukraine in video description

